In [1]:
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
tf.config.experimental.enable_tensor_float_32_execution(False)


2025-04-04 16:08:52.505533: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 16:08:52.514326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743779332.525695   39840 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743779332.529382   39840 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 16:08:52.540605: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Load MNIST
from tensorflow.keras.datasets import mnist

(x_train, labels_train), (x_val, labels_val) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.

x_train = x_train.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(labels_train, 10)
y_val = to_categorical(labels_val, 10)


In [3]:
# Load Nearly-MNIST from CSV
import pandas as pd

df_nearly = pd.read_csv('./datasets/nearly_mnist.csv')
x_nearly = df_nearly.iloc[:, :-1].values.astype('float32') / 255.
y_nearly_labels = df_nearly['Labels'].values.astype('int')

x_nearly = x_nearly.reshape(-1, 28, 28, 1)
y_nearly = to_categorical(y_nearly_labels, 10)


In [4]:
# Data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.1
)
datagen.fit(x_train)


In [5]:
# Define CNN
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2

inputs = Input(shape=x_train.shape[1:])
x = Conv2D(64, (5,5), activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = MaxPool2D((2, 2))(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = MaxPool2D((2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)
outputs = Dense(10, activation='softmax')(x)

net = Model(inputs=inputs, outputs=outputs)
net.summary()


I0000 00:00:1743779334.656459   39840 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6054 MB memory:  -> device: 0, name: NVIDIA RTX 2000 Ada Generation Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,605,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,685,770 (6.43 MB)

 Trainable params: 1,684,874 (6.43 MB)

 Non-trainable params: 896 (3.50 KB)

In [6]:
# Custom callback to evaluate on MNIST val and Nearly-MNIST each epoch
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

class ValidationAccLogger(Callback):
    def __init__(self, x_val, y_val, x_extra, y_extra):
        super().__init__()
        self.x_val = x_val
        self.y_val = y_val
        self.x_extra = x_extra
        self.y_extra = y_extra
        self.val_accuracies = []
        self.extra_accuracies = []

    def on_epoch_end(self, epoch, logs=None):
        val_acc = self.model.evaluate(self.x_val, self.y_val, verbose=0)[1]
        extra_acc = self.model.evaluate(self.x_extra, self.y_extra, verbose=0)[1]
        self.val_accuracies.append(val_acc)
        self.extra_accuracies.append(extra_acc)

val_logger = ValidationAccLogger(x_val, y_val, x_nearly, y_nearly)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [7]:
net.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

history = net.fit(
    datagen.flow(x_train, y_train, batch_size=128),
    validation_data=(x_val, y_val),
    epochs=20,
    callbacks=[early_stopping, val_logger]
)


Epoch 1/20


/home/bentengma/anaconda3/envs/tf218/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1743779336.303043   39983 service.cc:148] XLA service 0x7b9cac010800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743779336.303069   39983 service.cc:156]   StreamExecutor device (0): NVIDIA RTX 2000 Ada Generation Laptop GPU, Compute Capability 8.9
2025-04-04 16:08:56.352166: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743779336.491350   39983 cuda_dnn.cc:529] Loaded cuDNN version 90800


  8/469 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.1171 - loss: 8.7673

I0000 00:00:1743779338.646160   39983 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - accuracy: 0.4202 - loss: 6.7309 - val_accuracy: 0.1135 - val_loss: 11.1239
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.7847 - loss: 4.0654 - val_accuracy: 0.7141 - val_loss: 3.4123
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.8605 - loss: 2.7900 - val_accuracy: 0.8342 - val_loss: 2.2369
Epoch 4/20
201/469 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8924 - loss: 2.0336

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy (MNIST)')
plt.plot(val_logger.extra_accuracies, label='Validation Accuracy (Nearly-MNIST)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
inputs = Input(shape=x_train.shape[1:])
x = Conv2D(64, (5,5), activation='leakyrelu', padding='same')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = MaxPool2D((2, 2))(x)
x = Conv2D(128, (3,3), activation='leakyrelu', padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = MaxPool2D((2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='leakyrelu',)(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)
outputs = Dense(10, activation='softmax')(x)

net = Model(inputs=inputs, outputs=outputs)
net.summary()

In [ ]:
net.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

history = net.fit(
    datagen.flow(x_train, y_train, batch_size=128),
    validation_data=(x_val, y_val),
    epochs=20,
    callbacks=[early_stopping, val_logger]
)

plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy (MNIST)')
plt.plot(val_logger.extra_accuracies, label='Validation Accuracy (Nearly-MNIST)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()